In [1]:
# #@test {"skip": true}
# !pip install --quiet --upgrade tensorflow_federated_nightly
# !pip install --quiet --upgrade nest_asyncio

import nest_asyncio
nest_asyncio.apply()

In [32]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.utils import shuffle

tff.backends.reference.set_reference_context()

In [33]:
from tensorflow.keras import layers
from tensorflow.keras import activations

import math

In [34]:
mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()

In [35]:
[(x.dtype, x.shape) for x in mnist_train]

[(dtype('uint8'), (60000, 28, 28)), (dtype('uint8'), (60000,))]

In [36]:
NUM_EXAMPLES_PER_USER = 1000
BATCH_SIZE = 100
NUM_OF_CLIENTS = 2
# NUM_OF_CLIENTS_ARRAY = [0,0,0,0, 1,1,1,1, 2,2,2,2, 3,3,3,3, 4,4,4,4, 5,5,5,5, 6,6,6,6, 7,7,7,7, 8,8,8,8, 9,9,9,9]
# NUM_OF_CLIENTS_ARRAY = [0,1,2,3,4,5,6,7,8,9]

In [37]:
source_train_X, source_train_Y = shuffle(mnist_train[0], mnist_train[1], random_state=0)
source_test_X, source_test_Y = shuffle(mnist_test[0], mnist_test[1], random_state=0)

In [38]:
def get_data_for_digit_iid(source_X, source_Y):
    output_sequence = []
    for i in range(0, min(NUM_EXAMPLES_PER_USER, len(source_X)), BATCH_SIZE):
        batch_samples_X = source_X[i:i + BATCH_SIZE]
        batch_samples_Y = source_Y[i:i + BATCH_SIZE]
        output_sequence.append({
            'x': np.array([xx.flatten() / 255.0 for xx in batch_samples_X],dtype=np.float32),
            'y': np.array([yy for yy in batch_samples_Y], dtype=np.int32)
        })
    return output_sequence

xtr = int(len(source_train_X)/NUM_OF_CLIENTS)
ytr = int(len(source_train_Y)/NUM_OF_CLIENTS)

xte = int(len(source_test_X)/NUM_OF_CLIENTS)
yte = int(len(source_test_Y)/NUM_OF_CLIENTS)

federated_train_data = [get_data_for_digit_iid(source_train_X[(d*xtr):(d*xtr+xtr)], source_train_Y[(d*ytr):(d*ytr+ytr)]) for d in range(NUM_OF_CLIENTS)]

federated_test_data = [get_data_for_digit_iid(source_test_X[(d*xte):(d*xte+xte)], source_test_Y[(d*yte):(d*yte+yte)]) for d in range(NUM_OF_CLIENTS)]

In [39]:
def get_data_for_digit_niid(source, digit):
    output_sequence = []
    all_samples = [i for i, d in enumerate(source[1]) if d == digit]
    for i in range(0, min(len(all_samples), NUM_EXAMPLES_PER_USER), BATCH_SIZE):
        batch_samples = all_samples[i:i + BATCH_SIZE]
        output_sequence.append({
            'x': np.array([source[0][i].flatten() / 255.0 for i in batch_samples],dtype=np.float32),
            'y': np.array([source[1][i] for i in batch_samples], dtype=np.int32)
        })
    return output_sequence


# federated_train_data = [get_data_for_digit_niid(mnist_train, d) for d in NUM_OF_CLIENTS_ARRAY]

# federated_test_data = [get_data_for_digit_iid(source_test_X[(d*xte):(d*xte+xte)], source_test_Y[(d*yte):(d*yte+yte)]) for d in range(NUM_OF_CLIENTS)]

In [40]:
std_array = []
for client in federated_train_data:
    for batch in client:
        y = []
        for j in batch.get('y'):
            y.append(j)
    std_array.append(np.std(y))
# std_array = [{'weights': np.array(array, dtype=np.float32)}]
print(std_array)

[2.9112883745860696, 2.9645235704915556]


In [41]:
# mnist_train[0].shape

In [42]:
# federated_train_data

In [43]:
# federated_train_data[0][0].get('y').shape

In [44]:
# source_train_X.shape

In [45]:
# federated_train_data2[0][0].get('x').shape

In [46]:
# federated_train_data2[0][0].get('y').shape

In [47]:
BATCH_SPEC = collections.OrderedDict(
    x=tf.TensorSpec(shape=[None, 784], dtype=tf.float32),
    y=tf.TensorSpec(shape=[None], dtype=tf.int32))
BATCH_TYPE = tff.to_type(BATCH_SPEC)

str(BATCH_TYPE)

'<x=float32[?,784],y=int32[?]>'

In [48]:
LOCAL_DATA_TYPE = tff.SequenceType(BATCH_TYPE)

str(LOCAL_DATA_TYPE)

'<x=float32[?,784],y=int32[?]>*'

In [49]:
MODEL_SPEC = collections.OrderedDict(
    weights=tf.TensorSpec(shape=[784, 10], dtype=tf.float32),
    bias=tf.TensorSpec(shape=[10], dtype=tf.float32))
MODEL_TYPE = tff.to_type(MODEL_SPEC)

print(MODEL_TYPE)

<weights=float32[784,10],bias=float32[10]>


In [50]:
SERVER_MODEL_TYPE = tff.FederatedType(MODEL_TYPE, tff.SERVER)
CLIENT_DATA_TYPE = tff.FederatedType(LOCAL_DATA_TYPE, tff.CLIENTS)

In [51]:
# WEIGHT_SPEC = collections.OrderedDict(
#     weights = tf.TensorSpec(shape=[2], dtype=tf.float32))

In [52]:
WEIGHT_SPEC = tff.TensorType(
    dtype=tf.float32, shape=[1]
)
WEIGHT_TYPE = tff.to_type(WEIGHT_SPEC)

In [53]:
# print(WEIGHT_TYPE)

In [54]:
WEIGHT_DATA_TYPE = tff.FederatedType(WEIGHT_TYPE, tff.CLIENTS)

In [55]:
# tf.Tensor([2. 3. 4.], shape=(3,), dtype=float32)

Client

In [56]:
@tf.function
def forward_pass(model, batch):
    predicted_y = tf.nn.softmax(tf.matmul(batch['x'], model['weights']) + model['bias'])
    return -tf.reduce_mean(
        tf.reduce_sum(
            tf.one_hot(batch['y'], 10) * tf.math.log(predicted_y), axis=[1]))

@tff.tf_computation(MODEL_TYPE, BATCH_TYPE)
def batch_loss(model, batch):
    return forward_pass(model, batch)

In [57]:
@tff.tf_computation(MODEL_TYPE, BATCH_TYPE, tf.float32)
def batch_train(initial_model, batch, learning_rate):
    model_vars = collections.OrderedDict([
        (name, tf.Variable(name=name, initial_value=value))
        for name, value in initial_model.items()
    ])
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    
    @tf.function
    def _train_on_batch(model_vars, batch):
        # Perform one step of gradient descent using loss from `batch_loss`.
        with tf.GradientTape() as tape:
            loss = forward_pass(model_vars, batch)
        grads = tape.gradient(loss, model_vars)
        optimizer.apply_gradients(zip(tf.nest.flatten(grads), tf.nest.flatten(model_vars)))
        return model_vars
  
    return _train_on_batch(model_vars, batch)

In [58]:
def fun(x):
    print(x)

In [59]:
LOCAL_DATA_TYPE = tff.SequenceType(BATCH_TYPE)

@tff.federated_computation(MODEL_TYPE, tf.float32, LOCAL_DATA_TYPE)
def local_train(initial_model, learning_rate, all_batches):
    
    # Mapping function to apply to each batch.
    @tff.federated_computation(MODEL_TYPE, BATCH_TYPE)
    def batch_fn(model, batch):
        return batch_train(model, batch, learning_rate)
    return tff.sequence_reduce(all_batches, initial_model, batch_fn)

Server

In [60]:
@tff.federated_computation(MODEL_TYPE, LOCAL_DATA_TYPE)
def local_eval(model, all_batches):
    return tff.sequence_sum(
        tff.sequence_map(
            tff.federated_computation(lambda b: batch_loss(model, b), BATCH_TYPE),all_batches))

In [61]:
@tff.federated_computation(SERVER_MODEL_TYPE, CLIENT_DATA_TYPE)
def federated_eval(model, data):
    return tff.federated_mean(tff.federated_map(local_eval, [tff.federated_broadcast(model), data]))

In [62]:
SERVER_FLOAT_TYPE = tff.FederatedType(tf.float32, tff.SERVER)


@tff.federated_computation(SERVER_MODEL_TYPE, SERVER_FLOAT_TYPE,
                           CLIENT_DATA_TYPE, WEIGHT_DATA_TYPE)
def federated_train(model, learning_rate, data, std_array):
#     client_output =  tff.federated_map(local_train, [
#             tff.federated_broadcast(model),
#              tff.federated_broadcast(learning_rate), data
#         ])
    
#     print(client_output)

#     for i in range(len(client_output)):
#         client_output[i].get('weights') /= std_array[i]
    
    print("***********************")
    return tff.federated_mean(tff.federated_map(local_train, [
            tff.federated_broadcast(model),
             tff.federated_broadcast(learning_rate), data
        ]), std_array)

***********************


TypeError: The weight type {float32[1]}@CLIENTS is not a federated scalar.

In [ ]:
initial_model = collections.OrderedDict(
    weights=np.zeros([784, 10], dtype=np.float32),
    bias=np.zeros([10], dtype=np.float32))

In [ ]:
def modelR(weights, biases, data):
  count = 0
  avg = 0
  for j in range(len(federated_test_data[0])):
    l = [np.where(i==max(i))[0][0] for i in activations.sigmoid(np.matmul(federated_test_data[0][j].get('x'), model.get('weights'))).numpy()]
    Y = federated_test_data[0][j].get('y')
    
    for i in range(len(Y)):
      avg+=1
      if l[i] == Y[i]:
        count+= 1
  return count/avg

In [ ]:
model = initial_model
learning_rate = 0.1
for round_num in range(1):
    model = federated_train(model, learning_rate, federated_train_data, std_array)
    print(model)
    learning_rate = learning_rate * 0.9
    loss = federated_eval(model, federated_train_data)
    print('round {}, loss={}, accuracy={}'.format(round_num, loss, modelR(model.get('weights'), model.get('bias'), federated_test_data)))

In [ ]:
# accuracy = 0
# model = initial_model
# learning_rate = 0.1
# count = 0

# while accuracy<=0.95:
#     model = federated_train(model, learning_rate, federated_train_data)
#     learning_rate = learning_rate * 0.9
#     loss = federated_eval(model, federated_train_data)
#     accuracy = modelR(model.get('weights'), model.get('bias'), federated_test_data)
#     count+=1
    
# print('round {}, loss={}, accuracy={}, no. of cycles={}'.format(round_num, loss, accuracy, count))

In [ ]:
# def modelR(weights, biases, data):
#   count = 0
#   avg = 0
#   for j in range(len(federated_test_data[0])):
#     l = [np.where(i==max(i))[0][0] for i in activations.sigmoid(np.matmul(federated_test_data[0][j].get('x'), model.get('weights'))).numpy()]
#     Y = federated_test_data[0][j].get('y')
#     print('l={}, Y={}'.format(l, Y))
    
#     for i in range(len(Y)):
#       avg+=1
#       if l[i] == Y[i]:
#         count+= 1
#   return count/avg


In [ ]:
# modelR(model.get('weights'), model.get('bias'), federated_test_data)